In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

## LJ Speech

In [ ]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [ ]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("/path/to/pretrained_ljs.pth", net_g, None)

In [ ]:
stn_tst = get_text("VITS is Awesome!", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

## VCTK

In [2]:
hps = utils.get_hparams_from_file("./configs/vctk_base.json")

In [3]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model)#.cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("/fsx/home/1123/vits/vits/logs/vctk_base/G_250000.pth", net_g, None)

In [ ]:
stn_tst = get_text("VITS is Awesome!", hps)
with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])#.cuda()
    sid = torch.LongTensor([4])#.cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

### Voice Conversion

In [4]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)
print(hps.data.validation_files)

filelists/vctk_audio_sid_text_val_filelist.txt.cleaned


In [ ]:
print(dataset.audiopaths_sid_text[:3])

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([56])#.cuda()
    sid_tgt2 = torch.LongTensor([51])#.cuda()
    sid_tgt3 = torch.LongTensor([86])#.cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
_, _, _, _, y_gt, _, sid_gt = [x for x in data_list[1]]
print("GT SID: %d" % sid_gt.item())
ipd.display(ipd.Audio(y_gt[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
_, _, _, _, y_gt, _, sid_gt = [x for x in data_list[2]]
print("GT SID: %d" % sid_gt.item())
ipd.display(ipd.Audio(y_gt[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
_, _, _, _, y_gt, _, sid_gt = [x for x in data_list[3]]
print("GT SID: %d" % sid_gt.item())
ipd.display(ipd.Audio(y_gt[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))

In [5]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([56])#.cuda()
    sid_tgt2 = torch.LongTensor([51])#.cuda()
    sid_tgt3 = torch.LongTensor([86])#.cuda()
    audio1 = net_g.voice_conversion2(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    #audio2 = net_g.voice_conversion2(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    #audio3 = net_g.voice_conversion2(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
_, _, _, _, y_gt, _, sid_gt = [x for x in data_list[1]]
print("GT SID: %d" % sid_gt.item())
ipd.display(ipd.Audio(y_gt[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
#_, _, _, _, y_gt, _, sid_gt = [x for x in data_list[2]]
#print("GT SID: %d" % sid_gt.item())
#ipd.display(ipd.Audio(y_gt[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
#print("Converted SID: %d" % sid_tgt2.item())
#ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
#_, _, _, _, y_gt, _, sid_gt = [x for x in data_list[3]]
#print("GT SID: %d" % sid_gt.item())
#ipd.display(ipd.Audio(y_gt[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
#print("Converted SID: %d" % sid_tgt3.item())
#ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))

tensor([[[5.0035e+00, 1.8317e+02, 2.7468e+00, 3.4503e+00, 2.5843e+00,
          3.3634e+00, 4.1152e+00, 1.6647e+00, 3.3148e-01, 1.4319e+00,
          3.4257e+00, 6.0210e+00, 5.0544e-01, 1.8975e+00, 2.8115e+00,
          7.5125e+00, 5.6641e+00, 1.7957e+01, 1.0361e+01, 1.8444e+00,
          6.2422e-01, 3.2296e+00, 2.4709e+00, 3.2424e+00, 2.6783e+00,
          1.5907e+00, 3.0528e+00, 7.2487e-01, 1.0515e+01, 1.3924e-01,
          2.9225e+01, 4.5627e+00, 1.5078e+00, 6.8197e+00, 1.3574e+00,
          4.1445e+00, 2.6606e+00, 3.1850e-01, 2.8433e+00, 3.8999e-01,
          7.4687e+00, 1.6019e-01, 2.8416e+00, 2.5828e+01, 5.0869e+00,
          1.0717e+00, 2.4089e+00, 4.6618e+00, 2.4969e+00, 7.6846e-02,
          9.0006e+00, 3.0699e+01, 5.4119e+00, 2.9550e+00, 2.0808e+00,
          1.5853e+00, 1.3603e+01, 8.9406e-01, 8.9367e-01, 3.4121e+00,
          1.1207e+01, 1.4261e-01, 2.9390e+00, 1.5806e+00, 3.1461e-01,
          3.3663e-02, 3.9328e+00, 3.0452e+00, 5.8556e+00, 2.1701e+00,
          1.1762e+00

GT SID: 56


Converted SID: 56
